# Exploratory Notebook

### Exploring the Data

importing pandas with its alias pd

In [1]:
import pandas as pd
import CustomLibrary as cl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline
import seaborn as sns

SyntaxError: invalid syntax (CustomLibrary.py, line 382)

Unzipping the zipped data files (all at once)

In [ ]:
#!find . -name '*.tsv.gz' -exec gzip -d {} \;

Reading in all the data files to understand which ones will help answer our business questions. Checking the files with .head() and .tail() methods to view a section of the data. The .shape method will help tell how many rows we have in each data set. The .info() method will help discover the dtypes of the columns and possible missing values.

In [ ]:
df0= pd.read_csv('./data/zippedData/imdb.name.basics.csv')
df0.head()
#df0.shape

In [ ]:
df1= pd.read_csv('./data/zippedData/bom.movie_gross.csv')
df1.head()

In [ ]:
df2= pd.read_csv('./data/zippedData/imdb.title.akas.csv')
df2.head()

The dataset below gives us important information about the movies. Important columns: primary_title, start_year, and genres.

In [ ]:
df3= pd.read_csv('./data/zippedData/imdb.title.basics.csv')
df3.head()
df_title_movie = df3.rename(columns = {'primary_title':'movie'})
df_title_movie.head()

Changed the primary_title to 'movie' so that the merge with the next important data set goes smoothly. 

In [ ]:
df_title_movie.describe()

In [ ]:
df4= pd.read_csv('./data/zippedData/imdb.title.crew.csv')
df4.head()

In [ ]:
df5= pd.read_csv('./data/zippedData/imdb.title.principals.csv')
df5.head()

In [ ]:
df6= pd.read_csv('./data/zippedData/imdb.title.ratings.csv')
df6.head()

In [ ]:
df7= pd.read_csv('./data/zippedData/tmdb.movies.csv')
df7.head()

Found the next file to be one of the important files to gain an understanding of the costs and profits of different movies.

In [ ]:
df8= pd.read_csv('./data/zippedData/tn.movie_budgets.csv')
df8['start_year']= [int(x[-4:]) for x in df8['release_date']]
df8.head()
#df8.loc[df8['domestic_gross'] == '$0']

Created a new column called start_year that creates a year integer from the string from release_date. This is another qualifier for merging this dataset and the one chosen from above.

### Prepping Data

In [ ]:
def prep_Data(DataFrame1, DataFrame2):
    # We first need to match the column names for the movie titles to be able to join the two dataframes
    df_title_movie = DataFrame1.rename(columns = {'primary_title':'movie'})
    
    # Second: create a colum for start year as an integer type. This allows us join the dataframes by
    # the year of production so that we don't have errors with possible remakes.
    DataFrame2['start_year']= [int(x[-4:]) for x in DataFrame2['release_date']]
    
    # Third: merge the datasets on 'movie' and 'start_year'
    df_budget_merge = pd.merge(DataFrame2, df_title_movie, how ='inner', on = ('movie', 'start_year'))
    
    

### Cleaning the Data

First, merging the datasets so that we have all the information we need in one dataframe.
    The 'inner' join is chosen because each dataset has important data for the analysis. Not having both sides of the dataset makes the row useless in our chosen analysis. The merge is on 'movie' and 'start_year' to make sure that duplicates are at a minimum from having movies with the same name, but different release years.
    We also check for duplicates and drop the rows from our merged dataset.

In [ ]:
df_budget_merge = pd.merge(df8, df_title_movie, how ='inner', on = ('movie', 'start_year'))
df_duplicates = df_budget_merge[df_budget_merge['movie'].duplicated()]
#df_budget_merge.loc[df_budget_merge['domestic_gross'] == '$0'].head()
df_budget_merge.drop(df_duplicates.index, axis = 0, inplace = True)
#df_budget_merge.head()

We next check for missing values. We use worlwide_gross as a check to see if any data necessary for profit calculation is missing. If it is, we drop the row because filling the missing values would throw off our analysis and it does not account for more recently, stream only movies like from Netflix. We want to check only movies from the box office.

In [ ]:
df_no_values = df_budget_merge.loc[df_budget_merge['worldwide_gross'] == '$0']
df_budget_merge.drop(df_no_values.index, axis = 0, inplace = True)

Below is a function to change any string numbers into integers so we can use them for stats calculation.

In [ ]:
def clean_dollars(dataframe, column_str):
    dataframe[column_str] = dataframe[column_str].str.replace(',', '').str.replace('$', '').astype(int)
    return dataframe

clean_dollars(df_budget_merge, 'production_budget')
clean_dollars(df_budget_merge, 'domestic_gross')
clean_dollars(df_budget_merge, 'worldwide_gross')
df_budget_merge.info()

Next we create an advertisement column to add another important cost to producing a movie. Due to not having the data, we chose to follow the heuristic that advertisement budgets will generally be equal to production budgets. With the costs, we create a profit column from the differnce between the worldwide_gross column and the production_budget and advertisement_budget columns.

In [ ]:
df_budget_merge['advertisement_budget'] = df_budget_merge['production_budget']
df_budget_merge['profit'] = df_budget_merge['worldwide_gross'] - df_budget_merge['production_budget'] - df_budget_merge['advertisement_budget']
df_budget_merge.head(20)

We decide to sort the dataset by the profit values, with greatest profit being at the top.

In [ ]:
df_budget_merge.sort_values(by = ['profit'], axis = 0, ascending = False, inplace = True)

In [ ]:
def clean_Data(DataFrame):
    # Find any duplicates in the dataframe
    df_duplicates = DataFrame[DataFrame['movie'].duplicated()]
    
    # Drop the duplicates from the dataframe
    DataFrame.drop(df_duplicates.index, axis = 0, inplace = True)
    
    # Check for placeholder values by finding where worldwide_gross = '$0'
    df_no_values = DataFrame.loc[DataFrame['worldwide_gross'] == '$0']
    
    # Drop the placeholder values from the dataframe
    DataFrame.drop(df_no_values.index, axis = 0, inplace = True)
    
    # Change the number values represented by dtype String with dtype integers. Use clean_dollars function
    clean_dollars(DataFrame, 'production_budget')
    clean_dollars(DataFrame, 'domestic_gross')
    clean_dollars(DataFrame, 'worldwide_gross')
    
    # Create a new column for advertisement budget by using the production_budget column
    # Uses assumption that advertisement costs will generally cost the same as production
    DataFrame['advertisement_budget'] = DataFrame['production_budget']
    
    # Create a total costs column by adding the advertisement budget and the production budget
    df_budget_merge['total_costs'] = df_budget_merge['production_budget'] + df_budget_merge['advertisement_budget']
    
    # Create a profit column by taking the difference between the 'worldwide_gross' and 'total_costs' column
    DataFrame['profit'] = DataFrame['worldwide_gross'] - DataFrame['total_costs']
    
    # Sort the values by 'profit'
    DataFrame.sort_values(by = ['profit'], axis = 0, ascending = False, inplace = True)
    
    # Create a Return on Investment (ROI) percentage column by dividing profit and total costs multiplied by 100
    DataFrame['ROI'] = DataFrame['profit'] / DataFrame('total_costs') * 100
    
    # Seperate the different genres by using the indicator_str_parser function
    indicator_str_parser(df_budget_merge, 
                         'genres', 
                         ['Action', 'Adventure', 'Comedy', 'Drama', 'Family', 'Thriller', 'Documentary']
                        )
    
    # Create a budget category for low, mid, and high budgets based on the total_costs column
    DataFrame['budget_category'] = DataFrame['total_costs'].apply(lambda x: 
                                                                  'low' if x < 25000000 
                                                                  else 
                                                                  ('mid' if x < 100000000 
                                                                   else 'high' ))
    

### Using Genres

Next we check what type of genres are in each row.

In [ ]:
df_budget_merge.genres.value_counts()

We decide to use only certain common genres instead of the 214 possible collection of genres.

In [ ]:
# df_budget_merge['comedy_id'] = [1 if 'Comedy' in x
#                                 else 0 
#                                 for x in df_budget_merge['genres']]
# df_budget_merge['drama_id'] = [1 if 'Drama' in x
#                                 else 0 
#                                 for x in df_budget_merge['genres']]
# df_budget_merge['action_id'] = [1 if 'Action' in x
#                                 else 0 
#                                 for x in df_budget_merge['genres']]
# df_budget_merge.head()

We create a column to have a better understanding of the genres in each movie.

In [ ]:
#df_budget_merge['genre_tuple'] = list(zip(df_budget_merge['comedy_id'], df_budget_merge['drama_id'], df_budget_merge['action_id']))

In [ ]:
cl.indicator_str_parser(df_budget_merge, 'genres', ['Action', 'Adventure', 'Comedy', 'Documentary', 'Drama', 'Family', 'Thriller'])

WE next check to see which movies are missing from our previous genre categories.

In [ ]:
df_budget_merge.loc[df_budget_merge['genres_tuple'] == (0, 0, 0, 0, 0, 0, 0)]

In [ ]:
df_budget_merge.loc[df_budget_merge['genres_tuple'] == (0, 0, 0, 0, 0, 0, 0)].genres.value_counts()

In [ ]:
df_budget_merge.describe()

In [ ]:
df_budget_merge.info()

In [ ]:
df_budget_merge.genres_tuple.value_counts()

In [ ]:
df_budget_merge['ROI'] = df_budget_merge['profit'] / (df_budget_merge['production_budget'] + df_budget_merge['advertisement_budget']) * 100

In [ ]:
df_budget_merge['total_costs'] = df_budget_merge['production_budget'] + df_budget_merge['advertisement_budget']

In [ ]:
df_budget_merge.loc[df_budget_merge['ROI'] == df_budget_merge['ROI'].max()]

In [ ]:
df_budget_merge.describe()

## Visualizations

In [ ]:
#df_budget_merge['total_costs'].loc[df_budget_merge['total_costs'] < 200000000].hist(bins = 25)
#df_budget_merge['total_costs'].hist(bins = 100)
df_budget_merge['ROI'].loc[df_budget_merge['ROI'] < 2000].hist(bins = 20)
plt.show()

In [ ]:
#df_budget_merge.to_csv('Cleaned_Data.csv')

In [ ]:
df_budget_merge['budget_category'] = df_budget_merge['total_costs'].apply(lambda x: 'low' if x < 25000000 else ('mid' if x < 100000000 else 'high' ))

In [ ]:
df_budget_merge.head()

In [ ]:
df_budget_merge.loc[df_budget_merge['genres_Action_id'] == 1].ROI.median()

In [ ]:
genre_action = df_budget_merge.loc[df_budget_merge['genres_Action_id'] == 1]
genre_action['name'] = 'Action'
genre_adventure = df_budget_merge.loc[df_budget_merge['genres_Adventure_id'] == 1]
genre_adventure['name'] = 'Adventure'
genre_comedy = df_budget_merge.loc[df_budget_merge['genres_Comedy_id'] == 1]
genre_comedy['name'] = 'Comedy'
genre_drama = df_budget_merge.loc[df_budget_merge['genres_Drama_id'] == 1]
genre_drama['name'] = 'Drama'
genre_family = df_budget_merge.loc[df_budget_merge['genres_Family_id'] == 1]
genre_family['name'] = 'Family'
genre_thriller = df_budget_merge.loc[df_budget_merge['genres_Thriller_id'] == 1]
genre_thriller['name'] = 'Thriller'
genre_documentary = df_budget_merge.loc[df_budget_merge['genres_Documentary_id'] == 1]
genre_documentary['name'] = 'Documentary'
genres = [genre_action, genre_adventure, genre_comedy, genre_drama, genre_family, genre_thriller, genre_documentary]

In [ ]:

plt.scatter(df_budget_merge['total_costs'], df_budget_merge['ROI'])

plt.show()

### ATTEMPT 1 for Box Plots of Cost vs. ROI: Groupby

In [ ]:
df_budget_merge.groupby('genres_tuple')['total_costs'].mean().plot(kind = 'bar')

In [ ]:
#for x in ['Action', 'Adventure', 'Comedy', 'Drama', 'Family', 'Thriller', 'Documentary']
df_budget_merge.groupby('genres_tuple')['ROI'].median().plot(kind = 'bar')

### ATTEMPT 2 for Box Plots of Cost vs. ROI
#### Seperate Genres into Individual DataFrames

In [ ]:
fig = plt.figure()
for genre in genres:
    plt.bar(genre['name'], genre['total_costs'].mean())

In [ ]:
fig = plt.figure()
for genre in genres:
    plt.bar(genre['name'], genre['total_costs'].median())

In [ ]:
fig = plt.figure()
for genre in genres:
    plt.bar(genre['name'], genre['ROI'].mean())

In [ ]:
fig = plt.figure()
for genre in genres:
    plt.bar(genre['name'], genre['ROI'].median())

### Cleaning Up Attempt 2 Box Plots

In [ ]:
def Genres_DF(df, ls_of_genres):
    df_list = []
    for genre in ls_of_genres:
        'genre_' + genre = df.loc[df['genres_'+ genre +'_id'] == 1]
        df_list.append('genre_'+ genre)
    return df_list
        

We want to know how the Rate on Investment changes with genre. Or: How does genre affect the Rate on Investment of Movies? What genres have the best ROI?

In [ ]:

sns.set(font_scale = 3, style= 'whitegrid')
x1 = ['Action', 'Adventure', 'Comedy', 'Drama', 'Family', 'Thriller', 'Documentary']
y1 = []
for genre in genres:
#     x1.append(genre['name'])
    y1.append(genre['ROI'].mean())
fig = plt.figure(figsize=(20, 16))
ax = sns.barplot(x = x1, y = y1, color = 'blue')
ax.set(title = 'Mean Return on Investment by Genre', xlabel = 'Genres', ylabel = 'Return on Investment')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
#fig.savefig('Average_Return_on_Investment_by_Genre.png', bbox_inches = 'tight');

We see that the top 3 genres for ROI are: Adventure, Thriller, and Comedy. From this we know that the studio should start producing movies in one of these three genres.

We want to know how much it costs to produce movies of certain genres. What are the cheapest movies to produce by genre?

In [ ]:
sns.set_style('whitegrid')
x1 = ['Action', 'Adventure', 'Comedy', 'Drama', 'Family', 'Thriller', 'Documentary']
y1 = []
for genre in genres:
#     x1.append(genre['name'])
    y1.append(genre['total_costs'].median())
fig = plt.figure(figsize = (20, 16))
ax = sns.barplot(x = x1, y = y1, color = 'blue')
ax.set(title = 'Median Costs by Genre', xlabel = 'Genres', ylabel = 'Total Costs')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)
#fig.savefig('Median_Costs_by_Genre.png', bbox_inches = 'tight');

The cheapest movies to produce are Documentaries, Dramas, and Thrillers. Adventure, Action, and Family movies are the most expensive to produce. Taken in context of the previous graph, we should produce comedies and thrillers for cheapest investment and highest ROI. If the studio is willing to gamble, Adventure would bring in the highest profit. It takes a lot of investment, but is most likely to do well in the box office.

In [ ]:
action = df_budget_merge[df_budget_merge['genres_Action_id'] == 1]
adventure = df_budget_merge[df_budget_merge['genres_Adventure_id'] == 1]
comedy = df_budget_merge[df_budget_merge['genres_Comedy_id'] == 1]
family = df_budget_merge[df_budget_merge['genres_Family_id'] == 1]
thriller = df_budget_merge[df_budget_merge['genres_Thriller_id'] == 1]
documentary = df_budget_merge[df_budget_merge['genres_Documentary_id'] == 1]
other = df_budget_merge[df_budget_merge['genres_tuple'] == '(0, 0, 0, 0, 0, 0, 0)']
drama = df_budget_merge[df_budget_merge['genres_Drama_id'] == 1]

In [ ]:
fig = plt.figure()
ax = sns.barplot(x = x1, y = y1, color = 'blue')

In [ ]:
new_df = pd.DataFrame(columns = ['genre_budget', 'genre', 'ROI', 'budget_category'])
new_df

In [ ]:
new_df['ROI'] = [df_budget_merge.loc[(df_budget_merge['genres_Action_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Action_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Action_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Adventure_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Adventure_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Adventure_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Comedy_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Comedy_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Comedy_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Drama_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Drama_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Drama_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Family_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Family_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Family_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Thriller_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Thriller_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Thriller_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_Documentary_id'] == 1) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Documentary_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_Documentary_id'] == 1) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median(),
                df_budget_merge.loc[(df_budget_merge['genres_not_parsed_id'] == 0) & 
                                    (df_budget_merge['genres_tuple'] == (0, 0, 0, 0, 0, 0, 0)) & 
                                    (df_budget_merge['total_costs'] >= 100000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_not_parsed_id'] == 0) & 
                                    (df_budget_merge['genres_tuple'] == (0, 0, 0, 0, 0, 0, 0)) & 
                                    (df_budget_merge['total_costs'] < 100000000) & 
                                    (df_budget_merge['total_costs'] >= 25000000)].ROI.median(), 
                df_budget_merge.loc[(df_budget_merge['genres_not_parsed_id'] == 0) & 
                                    (df_budget_merge['genres_tuple'] == (0, 0, 0, 0, 0, 0, 0)) & 
                                    (df_budget_merge['total_costs'] < 25000000)].ROI.median()]
new_df['budget_category'] = ['high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low',
                            'high', 'mid', 'low']
new_df['genre'] = ['Action', 'Action', 'Action', 'Adventure', 'Adventure', 'Adventure', 'Comedy', 'Comedy', 'Comedy',
                  'Drama', 'Drama', 'Drama', 'Family', 'Family', 'Family', 'Thriller', 'Thriller', 'Thriller', 
                  'Documentary', 'Documentary', 'Documentary', 'Other', 'Other', 'Other']



In [ ]:
new_df

In [ ]:
fig = plt.figure(figsize= (20, 16))
ax = sns.barplot(data = new_df, x = 'genre', y = 'ROI', hue = 'budget_category', palette=sns.color_palette(['lightblue', 'steelblue', 'royalblue']), hue_order = ['low', 'mid', 'high'])
ax.set(title = 'Median ROI for Genre Categorized by Budget Tier', xlabel = 'Genres', ylabel = 'Return on Investment')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
ax.legend(title = 'Budget Tier')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
#fig.savefig('Median_ROI_for_Genre_Categorized_by_Budget_Tier.png', bbox_inches = 'tight');

In [ ]:
fig = plt.figure(figsize= (20, 16))
ax = sns.barplot(data = new_df, x = 'ROI', y = 'genre', hue = 'budget_category', palette=sns.color_palette(['lightblue', 'steelblue', 'royalblue']), hue_order = ['low', 'mid', 'high'], orient = 'h')
ax.set(title = 'Median ROI for Genre Categorized by Budget Tier', ylabel = 'Genres', xlabel = 'Return on Investment')
ax.legend(title = 'Budget Tier')
ax.xaxis.set_major_formatter(mtick.PercentFormatter());

In [ ]:
def Low_Budget_Genres(DataFrame):
    # Create a list for rows of data
    r = []
    # Iterate through desired genres
    for x in ['Action', 'Adventure', 'Comedy', 'Documentary', 'Drama', 'Family', 'Thriller']:
        # Append  a list of [ROI, Genre] as a row to your list of rows. ROI should be for low budget films only
        r.append([DataFrame.loc[(DataFrame[f'genres_{x}_id'] == 1) & 
                                (DataFrame['total_costs'] < 25000000)].ROI.median(), x])
    # Create a new DataFrame for your list of rows with columns of Median ROI and Genre
    new_dF = pd.DataFrame(r, columns = ('Median ROI', 'Genre'))
    # Return your new DataFrame
    return new_dF
        

In [ ]:
lb_Genres = Low_Budget_Genres(df_budget_merge)
lb_Genres

In [ ]:
def Mid_Budget_Genres(DataFrame):
    # Create a list for rows of data
    r = []
    # Iterate through desired genres
    for x in ['Action', 'Adventure', 'Comedy', 'Documentary', 'Drama', 'Family', 'Thriller']:
        # Append  a list of [ROI, Genre] as a row to your list of rows. ROI should be for mid budget films only
        r.append([DataFrame.loc[(DataFrame[f'genres_{x}_id'] == 1) & 
                                (DataFrame['total_costs'] >= 25000000) & 
                                (DataFrame['total_costs'] < 100000000)].ROI.median(), x])
    # Create a new DataFrame for your list of rows with columns of Median ROI and Genre
    new_dF = pd.DataFrame(r, columns = ('Median ROI', 'Genre'))
    # Return your new DataFrame
    return new_dF


In [ ]:
mb_Genres = Mid_Budget_Genres(df_budget_merge)
mb_Genres

In [ ]:
def High_Budget_Genres(DataFrame):
    # Create a list for rows of data
    r = []
    # Iterate through desired genres
    for x in ['Action', 'Adventure', 'Comedy', 'Documentary', 'Drama', 'Family', 'Thriller']:
        # Append  a list of [ROI, Genre] as a row to your list of rows. ROI should be for high budget films only
        r.append([DataFrame.loc[(DataFrame[f'genres_{x}_id'] == 1) & 
                                (DataFrame['total_costs'] > 100000000)].ROI.median(), x])
    # Create a new DataFrame for your list of rows with columns of Median ROI and Genre
    new_dF = pd.DataFrame(r, columns = ('Median ROI', 'Genre'))
    # Return your new DataFrame
    return new_dF


In [ ]:
hb_Genres = High_Budget_Genres(df_budget_merge)
hb_Genres

In [ ]:
def LB_Genres_Graph(DataFrame):
    fig = plt.figure(figsize= (20, 16))
    ax = sns.barplot(data = DataFrame, x = 'Genre', y = 'Median ROI', color = 'lightblue')
    ax.set(title = 'Median ROI for Low Budget Films by Genre', xlabel = 'Genres', ylabel = 'Return on Investment')
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    #fig.savefig('Median_ROI_for_Low_Budget_Films_by_Genre.png', bbox_inches = 'tight');
    return plt.show()

In [ ]:
def MB_Genres_Graph(DataFrame):
    fig = plt.figure(figsize= (20, 16))
    ax = sns.barplot(data = DataFrame, x = 'Genre', y = 'Median ROI', color = 'steelblue')
    ax.set(title = 'Median ROI for Mid Bidget Films by Genre', xlabel = 'Genres', ylabel = 'Return on Investment')
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    #fig.savefig('Median_ROI_for_Mid_Budget_Films_by_Genre.png', bbox_inches = 'tight');
    return plt.show()

In [ ]:
def HB_Genres_Graph(DataFrame):
    fig = plt.figure(figsize= (20, 16))
    ax = sns.barplot(data = DataFrame, x = 'Genre', y = 'Median ROI', color = 'blue')
    ax.set(title = 'Median ROI for High Budget Film by Genre', xlabel = 'Genres', ylabel = 'Return on Investment')
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
    #fig.savefig('Median_ROI_for_High_Budget_Films_by_Genre.png', bbox_inches = 'tight');
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())

In [ ]:
import code.visualizations as viz

In [ ]:
LB_Genres_Graph(lb_Genres)

In [ ]:
MB_Genres_Graph(mb_Genres)

In [ ]:
HB_Genres_Graph(hb_Genres)

### Question 4- How does the movie budget affect the ROI by Genre?

When trying to find the best genre to produce a movie, it is important to know how each genre does with different budgets. This is done by first making a dataset out of clean dataset with genres and the median ROI. The median ROI was chosen so that any outliers would not affect the graph. To find the right data to put into the dataframe, 

## Evaluation 

The overall business problem that was given asked how a new movie studio may be as successful as possible. By looking more deeply into the data on movies from the past 10 years, the analysis has shown what types of genres have done well with certain sized budgets. This successfully has answered the main question in a basic. This analysis is helpful as done, however there are certain areas that could bring more accuracy to the analysis. 

    -The Advertisement Cost: In the analysis, it was assumed that advertisement budget was equal to the production budget. However, it unlikely that all movies had an equal advertisement and production budget. By using the    exact advertisement budget, it will increase the accuracy of the ROI.
    -Inflation: This analysis did not account for inflation. While only over a 10 year period, the inflation would have an effect on the money values of the movies, which would increase the accuracy of the comparison between  movies.
    -Time Period of Analysis: As mentioned before, the dataframes were only for movies from the last 10 years.     While useful for immediate future movies, having a longer period of data could increase how robust our analysis is.
    -Box Office Only: This data only takes into account of the box office. If Microsoft desired to go into         streaming services and produce movies for the streaming service only, it would be more accurate to include data on streamed movies.

## Conclusion

This analysis brings forward two recommendations:

**Produce high budget adventure films-** if the studio has a high production & advertisement budget, adventure films had the highest median ROI for high budgets

**Produce low budget thrillers-** if the studio has a low budget, thrillers have the highest possible ROI of any genre

## Next Steps

**Evaluating personnel that maximize ROI-** By analyzing the personnel of successful movies in genres such as thrillers and adventures, the studio could attempt to hire those that would most likely help produce a movie with a high ROI.

**Find accurate advertisement budgets for movies and account for inflation-** By going back into the analysis, the accuracy of the ROIs could be increased.

**Constructing Portfolios of Films-** With a budget, a portfolio of films could be maximized for ROI by genre and budget.